In [128]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl

from sklearn.ensemble import RandomForestRegressor,RandomForestClassifier
import time
from sklearn import preprocessing
from sklearn.model_selection import train_test_split, GridSearchCV

np.random.RandomState(seed=2019)

In [129]:
#matplotlib inline
plt.rcParams['figure.figsize'] = 16,6
plt.rcParams['xtick.color'] = 'w'  
plt.rcParams['ytick.color'] = 'w'  
mpl.style.use('ggplot')

## 数据导入和处理

In [130]:
train_df_path = r'../data/processed_data/train_df.csv'
test_df_path = r'../data/processed_data/test_df.csv'

In [131]:
train_df = pd.read_csv(train_df_path,index_col=0) #第一列作为index
test_df = pd.read_csv(test_df_path,index_col=0)

In [132]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20631 entries, 0 to 20630
Data columns (total 28 columns):
id            20631 non-null int64
cycle         20631 non-null int64
setting1      20631 non-null float64
setting2      20631 non-null float64
setting3      20631 non-null float64
s1            20631 non-null float64
s2            20631 non-null float64
s3            20631 non-null float64
s4            20631 non-null float64
s5            20631 non-null float64
s6            20631 non-null float64
s7            20631 non-null float64
s8            20631 non-null float64
s9            20631 non-null float64
s10           20631 non-null float64
s11           20631 non-null float64
s12           20631 non-null float64
s13           20631 non-null float64
s14           20631 non-null float64
s15           20631 non-null float64
s16           20631 non-null float64
s17           20631 non-null float64
s18           20631 non-null float64
s19           20631 non-null float64
s20    

In [133]:
train_df.head()

,id,cycle,setting1,setting2,setting3,s1,s2,s3,s4,s5,...,s14,s15,s16,s17,s18,s19,s20,s21,RUL,cycle_norm
0,1,1,0.459770,0.166667,0.0,0.0,0.183735,0.406802,0.309757,0.0,...,0.199608,0.363986,0.0,0.333333,0.0,0.0,0.713178,0.724662,130,0.00000
1,1,2,0.609195,0.250000,0.0,0.0,0.283133,0.453019,0.352633,0.0,...,0.162813,0.411312,0.0,0.333333,0.0,0.0,0.666667,0.731014,130,0.00277
2,1,3,0.252874,0.750000,0.0,0.0,0.343373,0.369523,0.370527,0.0,...,0.171793,0.357445,0.0,0.166667,0.0,0.0,0.627907,0.621375,130,0.00554
3,1,4,0.540230,0.500000,0.0,0.0,0.343373,0.256159,0.331195,0.0,...,0.174889,0.166603,0.0,0.333333,0.0,0.0,0.573643,0.662386,130,0.00831
4,1,5,0.390805,0.333333,0.0,0.0,0.349398,0.257467,0.404625,0.0,...,0.174734,0.402078,0.0,0.416667,0.0,0.0,0.589147,0.704502,130,0.01108


In [134]:
test_df.head()

,id,cycle,setting1,setting2,setting3,s1,s2,s3,s4,s5,...,s14,s15,s16,s17,s18,s19,s20,s21,cycle_norm,RUL
0,1,1,0.632184,0.750000,0.0,0.0,0.545181,0.310661,0.269413,0.0,...,0.132160,0.308965,0.0,0.333333,0.0,0.0,0.558140,0.661834,0.00000,142
1,1,2,0.344828,0.250000,0.0,0.0,0.150602,0.379551,0.222316,0.0,...,0.204768,0.213159,0.0,0.416667,0.0,0.0,0.682171,0.686827,0.00277,141
2,1,3,0.517241,0.583333,0.0,0.0,0.376506,0.346632,0.322248,0.0,...,0.155640,0.458638,0.0,0.416667,0.0,0.0,0.728682,0.721348,0.00554,140
3,1,4,0.741379,0.500000,0.0,0.0,0.370482,0.285154,0.408001,0.0,...,0.170090,0.257022,0.0,0.250000,0.0,0.0,0.666667,0.662110,0.00831,139
4,1,5,0.580460,0.500000,0.0,0.0,0.391566,0.352082,0.332039,0.0,...,0.152751,0.300885,0.0,0.166667,0.0,0.0,0.658915,0.716377,0.01108,138


In [135]:
X_train = train_df.iloc[:,5:26]
y_train = train_df.iloc[:,-2]
print(X_train.head())
X_train = np.array(X_train)
y_train = np.array(y_train)

    s1        s2        s3        s4   s5   s6        s7        s8        s9  \
0  0.0  0.183735  0.406802  0.309757  0.0  1.0  0.726248  0.242424  0.109755   
1  0.0  0.283133  0.453019  0.352633  0.0  1.0  0.628019  0.212121  0.100242   
2  0.0  0.343373  0.369523  0.370527  0.0  1.0  0.710145  0.272727  0.140043   
3  0.0  0.343373  0.256159  0.331195  0.0  1.0  0.740741  0.318182  0.124518   
4  0.0  0.349398  0.257467  0.404625  0.0  1.0  0.668277  0.242424  0.149960   

   s10    ...          s12       s13       s14       s15  s16       s17  s18  \
0  0.0    ...     0.633262  0.205882  0.199608  0.363986  0.0  0.333333  0.0   
1  0.0    ...     0.765458  0.279412  0.162813  0.411312  0.0  0.333333  0.0   
2  0.0    ...     0.795309  0.220588  0.171793  0.357445  0.0  0.166667  0.0   
3  0.0    ...     0.889126  0.294118  0.174889  0.166603  0.0  0.333333  0.0   
4  0.0    ...     0.746269  0.235294  0.174734  0.402078  0.0  0.416667  0.0   

   s19       s20       s21  
0  0.0  0

In [136]:
X_test = test_df.iloc[:,5:26]
y_test = test_df.iloc[:,-1]#注意，当前用的数据中测试集的RUL是最后一列!!!
print(X_test.head())
X_test = np.array(X_test)
y_test = np.array(y_test)

    s1        s2        s3        s4   s5   s6        s7        s8        s9  \
0  0.0  0.545181  0.310661  0.269413  0.0  1.0  0.652174  0.212121  0.127614   
1  0.0  0.150602  0.379551  0.222316  0.0  1.0  0.805153  0.166667  0.146684   
2  0.0  0.376506  0.346632  0.322248  0.0  1.0  0.685990  0.227273  0.158081   
3  0.0  0.370482  0.285154  0.408001  0.0  1.0  0.679549  0.196970  0.105717   
4  0.0  0.391566  0.352082  0.332039  0.0  1.0  0.694042  0.166667  0.102396   

   s10    ...          s12       s13       s14       s15  s16       s17  s18  \
0  0.0    ...     0.646055  0.220588  0.132160  0.308965  0.0  0.333333  0.0   
1  0.0    ...     0.739872  0.264706  0.204768  0.213159  0.0  0.416667  0.0   
2  0.0    ...     0.699360  0.220588  0.155640  0.458638  0.0  0.416667  0.0   
3  0.0    ...     0.573561  0.250000  0.170090  0.257022  0.0  0.250000  0.0   
4  0.0    ...     0.737740  0.220588  0.152751  0.300885  0.0  0.166667  0.0   

   s19       s20       s21  
0  0.0  0

RF不需要归一化

In [137]:
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(20631, 21) (20631,) (13096, 21) (13096,)


现在X_train, y_train, X_val, y_val, X_test, y_test已经准备好了

## 模型构建

random forest

In [138]:
#model = RandomForestRegressor()
model = RandomForestClassifier()

## 模型训练

In [139]:
params = {
    'n_estimators': range(10, 20, 1)
    'max_depth': range(10,100,9)
}

gs = GridSearchCV(model,params,cv=4,verbose=1)

SyntaxError: invalid syntax (<ipython-input-139-89f78bda3275>, line 3)

In [ ]:
gs.fit(X_train,y_train)

#查验优化后的超参数配置
print(gs.best_score_)
print(gs.best_params_)

## 模型评估

In [ ]:
y_pred = gs.predict(X_test)

In [ ]:
def rmse(predictions,targets):
    return np.sqrt(((predictions-targets)**2).mean())

In [ ]:
my_rmse = rmse(y_pred,y_test)

In [ ]:
print("Test rmse: {}".format(my_rmse))